# Raster Vectorization Project

In [12]:
import numpy as np
import skimage.io
import os
import random
from matplotlib import pylab
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

filename = 'C:/Users/Ashley/Documents/GitHub/image-rgb-in-3D/gradientbox.png'
img = skimage.io.imread( filename )

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Ashley/Documents/GitHub/image-rgb-in-3D/gradientbox.png'

In [ ]:
%matplotlib inline
%store img

In [ ]:
%lsmagic

In [ ]:
plt.imshow(img)

# Convert Image Array to 6D with X, Y, R, G, B, A

In [ ]:
height, width, depth = img.shape
# print(img.shape)
# np.meshgrid( width, height )
# print(np.meshgrid(width, height))
# imageArray = np.append( np.meshgrid( width, height ), img )
# print(imageArray)


heightAr = list(range(0, height))
# print(heightAr)
widthAr = list(range(0, width))
# print(widthAr)
meshGrid = np.meshgrid(widthAr, heightAr)
imageArray = np.concatenate((meshGrid[0][:, :, None], meshGrid[1][:, :, None], img), axis = 2)
# print(imageArray[0, 2])
# print(imageArray.shape)

# reshaped is the input for the plane-fitting
reshaped = imageArray.reshape(width * height, -1)
print(reshaped)




In [ ]:

# print(reshaped.shape)

# xyg = reshaped[:, [0, 1, 3]]
# print(xyg)

# prints out the channel that is wanted with the xy coordinates
def getChannel(imageArray, channel):
    xyc = [0, 1, channel]
    return imageArray[:, xyc]

# channel constants
r = 2
g = 3
b = 4
a = 5

print (getChannel(reshaped, b))
# noAlpha = reshaped[:, :5]
# print(noAlpha)

# Plane fitting algorithm
[Source Code on GitHub](https://gist.github.com/lambdalisue/7201028) 


In [ ]:
# https://gist.github.com/lambdalisue/7201028

def standard_fit(X):
    """
    Find (n - 1) dimensional standard (e.g. line in 2 dimension, plane in 3
    dimension, hyperplane in n dimension) via solving Singular Value
    Decomposition.
    The idea was explained in the following references
    - http://www.caves.org/section/commelect/DUSI/openmag/pdf/SphereFitting.pdf
    - http://www.geometrictools.com/Documentation/LeastSquaresFitting.pdf
    - http://www.ime.unicamp.br/~marianar/MI602/material%20extra/svd-regression-analysis.pdf
    - http://www.ling.ohio-state.edu/~kbaker/pubs/Singular_Value_Decomposition_Tutorial.pdf
    Example:
        >>> XY = [[0, 1], [3, 3]]
        >>> XY = np.array(XY)
        >>> C, N = standard_fit(XY)
        >>> C
        array([ 1.5,  2. ])
        >>> N
        array([-0.5547002 ,  0.83205029])
    Args:
        X: n x m dimensional matrix which n indicate the number of the dimension
            and m indicate the number of points
    Returns:
        [C, N] where C is a centroid vector and N is a normal vector
    """
    # Find the average of points (centroid) along the columns
    C = np.average(X, axis=0)
    # Create CX vector (centroid to point) matrix
    CX = X - C
    # Singular value decomposition
    U, S, V = np.linalg.svd(CX)
    # The last row of V matrix indicate the eigenvectors of
    # smallest eigenvalues (singular values).
    N = V[-1]
    return C, N

def projection(x, C, N):
    """
    Create orthogonal projection matrix of x on the plane
    Args:
        x: n x m dimensional matrix
        C: n dimensional vector whicn indicate the centroid of the standard
        N: n dimensional vector which indicate the normal vector of the standard
    Returns:
        n x m dimensional matrix which indicate the orthogonal projection points
        on the plane
    """
    rows, cols = x.shape
    NN = np.tile(N, (rows, 1))
    D = distance(x, C, N)
    DD = np.tile(D, (cols, 1)).T
    return x - DD * NN

def distance(x, C, N):
    """
    Calculate an orthogonal distance between the points and the standard
    Args:
        x: n x m dimensional matrix
        C: n dimensional vector whicn indicate the centroid of the standard
        N: n dimensional vector which indicate the normal vector of the standard
    Returns:
        m dimensional vector which indicate the orthogonal disntace. the value
        will be negative if the points beside opposite side of the normal vector
    """
    return np.dot(x-C, N)

def function(x, C, N):
    """
    Calculate an orthogonal projection of the points on the standard
    Args:
        x: (n-1) x m dimensional matrix
        C: n dimensional vector whicn indicate the centroid of the standard
        N: n dimensional vector which indicate the normal vector of the standard
    Returns:
        m dimensional vector which indicate the last attribute value of
        orthogonal projection
    """
    Ck = C[0:-1]    # centroid for known parameters
    Nk = N[0:-1]    # normal for known parmeters
    Cu = C[-1]      # centroid for unknown parameter
    Nu = N[-1]      # normal for unknown parameter
    return np.dot(x-Ck, Nk) * -1.0 / Nu + Cu

#===============================================================================
#
# Unittest
#
#===============================================================================

In [ ]:
def multi(default_runs=1):
    def inner(fn):
#         @wraps(fn)
        def wrapper(*args, **kwargs):
            for i in range(int(config.get('runs', default_runs))):
                fn(*args, **kwargs)
        return wrapper
    return inner

def multi_dimensional(dimension=9):
    def inner(fn):
        for d in range(1, dimension+1):
#             @wraps(fn)
            def wrapper(d=d, *args, **kwargs):
                fn(*args, dimension=d, **kwargs)
            wrapper.__name__ += "_%dd" % d
            globals()[wrapper.__name__] = wrapper
    return inner

def cosine(a, b):
    """
    Calculate cosine similarity of two vectors
    Args:
        a: n dimensional vector A
        b: n dimensional vector B
    Returns:
        cosine value which 1, -1, and 0 indicates a) completely same direction,
        b) completely opposite direction, and c) orthgonal direction
        respectively.
    """
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    return np.dot(a, b) / (norma * normb)

@multi_dimensional(9)
@multi(10)
# @attr(matplotlib=False)
def test_standard_fit(dimension):
    # define true parameters and function
    N = np.random.rand(dimension)
    N = N / np.linalg.norm(N)
    C = np.random.rand(dimension)
    F = partial(standard_fit.function, C=C, N=N)
    E = (np.random.rand(50) - 0.5) * 0.01
    NN = np.tile(N, (50, 1))
    EE = np.tile(E, (dimension, 1)).T

    # create corresponding points with some error
    e = np.random.normal(size=50) * 0.01
    xy = np.random.rand(50, dimension-1)
    xyz = np.c_[xy, F(xy) + e]
    # shift xyz along N vector
    xyz = xyz + EE * NN

    # run standard_fit
    c, n = standard_fit.standard_fit(xyz)
    f = partial(standard_fit.function, C=c, N=n)

    # normal vector similarity check
    similarity_n = cosine(n, N)
    ok_(abs(similarity_n) > 0.99,
            "similarity_n (%f) is lower than threshold" % abs(similarity_n))

@multi_dimensional(9)
@multi(10)
# @attr(matplotlib=False)
def test_function(dimension):
    # define true parameters and function
    N = np.random.rand(dimension)
    N = N / np.linalg.norm(N)
    C = np.random.rand(dimension)
    F = partial(standard_fit.function, C=C, N=N)

    # create corresponding points with some error
    xy = np.random.rand(50, dimension-1)
    xyz = np.c_[xy, F(xy)]

    # run standard_fit
    c, n = standard_fit.standard_fit(xyz)
    f = partial(standard_fit.function, C=c, N=n)

    xyz_signal = np.c_[xy, F(xy)]
    xyz_fitted = np.c_[xy, f(xy)]

    np.testing.assert_array_almost_equal(
            xyz_signal, xyz_fitted, decimal=6)

@multi_dimensional(9)
@multi(10)
# @attr(matplotlib=False)
def test_distance(dimension):
    # define true parameters and function
    N = np.random.rand(dimension)
    N = N / np.linalg.norm(N)
    C = np.random.rand(dimension)
    F = partial(standard_fit.function, C=C, N=N)
    E = (np.random.rand(50) - 0.5) * 0.01
    NN = np.tile(N, (50, 1))
    EE = np.tile(E, (dimension, 1)).T

    # create corresponding points with some error
    xy = np.random.rand(50, dimension-1)
    xyz = np.c_[xy, F(xy)]
    # shift xyz along N vector
    xyz = xyz + EE * NN

    # run standard_fit
    c, n = standard_fit.standard_fit(xyz)

    D = standard_fit.distance(xyz, C, N)
    np.testing.assert_array_almost_equal(
            D, E, decimal=6)

    d = standard_fit.distance(xyz, c, n)
    np.testing.assert_array_almost_equal(
            D, d, decimal=1)

@multi_dimensional(9)
@multi(10)
# @attr(matplotlib=False)
def test_projection(dimension):
    # define true parameters and function
    N = np.random.rand(dimension)
    N = N / np.linalg.norm(N)
    C = np.random.rand(dimension)
    F = partial(standard_fit.function, C=C, N=N)
    E = (np.random.rand(50) - 0.5) * 0.01
    NN = np.tile(N, (50, 1))
    EE = np.tile(E, (dimension, 1)).T

    # create corresponding points with some error
    xy = np.random.rand(50, dimension-1)
    xyz = np.c_[xy, F(xy)]
    # shift xyz along N vector
    xyz = xyz + EE * NN

    # run standard_fit
    c, n = standard_fit.standard_fit(xyz)

    projected = standard_fit.projection(xyz, c, n)

    PC = projected - c
    for i, pc in enumerate(PC):
        dot = np.dot(pc, n)
        np.testing.assert_almost_equal(dot, 0)

    distances = standard_fit.distance(xyz, c, n)
    for x, p, d in zip(xyz, projected, distances):
        norm = np.linalg.norm(p - x)
        np.testing.assert_almost_equal(norm, abs(d))

try:
    import matplotlib.pyplot as pl
    from mpl_toolkits.mplot3d import Axes3D
 
    def extend_plt3d(plt3d):
        def draw_vector(p1, p2, *args, **kwargs):
            from matplotlib.patches import FancyArrowPatch
            from mpl_toolkits.mplot3d import proj3d
            class Arrow3D(FancyArrowPatch):
                def __init__(self, xs, ys, zs, *args, **kwargs):
                    FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
                    self._verts3d = xs, ys, zs
                    self._verts3d = [x.flatten() for x in self._verts3d]
                def draw(self, renderer):
                    xs3d, ys3d, zs3d = self._verts3d
                    xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
                    self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
                    FancyArrowPatch.draw(self, renderer)
            xs = np.array([p1[0], p2[0]])
            ys = np.array([p1[1], p2[1]])
            zs = np.array([p1[2], p2[2]])
            plt3d.add_artist(Arrow3D(xs, ys, zs, *args,
                mutation_scale=20, arrowstyle="-|>", **kwargs))
        setattr(plt3d, 'vector', draw_vector)
        return plt3d
 
    def create_vector(top, tail=(0, 0, 0)):
        return np.array([tail, top])

    def rotate(vector, normal, theta):
        x, y, z = normal
        R = np.array([
            [0, -z, y],
            [z, 0, -x],
            [-y, x, 0],
        ])
        I = np.identity(3)
        M = I + np.sin(theta) * R + (1 - np.cos(theta)) * np.dot(R, R)
        return np.dot(M, vector)

#     @attr(matplotlib=True)
    def test_plot_standard_plane():
        scale = 10
        # define true parameters and function
        O = np.array([0, 0, 0])
        N = np.random.rand(3)
        N = N / np.linalg.norm(N)
        C = np.random.rand(3)
        F = partial(standard_fit.function, C=C, N=N)
        E = (np.random.rand(50) - 0.5) * scale / 2
        NN = np.tile(N, (50, 1))
        EE = np.tile(E, (3, 1)).T

        # create corresponding points with some error
        xy = np.random.rand(50, 2) * scale
        xyz = np.c_[xy, F(xy)]
        # shift xyz along N vector
        xyz = xyz + EE * NN

        # run standard_fit
        c, n = standard_fit.standard_fit(xyz)

        # projection
        prj = standard_fit.projection(xyz, c, n)

        # find farthest point
        norms = np.apply_along_axis(np.linalg.norm, 1, prj - c)
        ind = np.argmax(norms)
        farthest = prj[ind]

        # find surface minimum and maximum value of each axis
        p1 = rotate(farthest - c, n, np.pi/4) + c
        ps = np.array([
            p1,
            rotate(p1 - c, n, np.pi/2) + c,
            rotate(p1 - c, n, np.pi) + c,
            rotate(p1 - c, n, -np.pi/2) + c,
        ])
        pmin = np.min(ps, axis=0)
        pmax = np.max(ps, axis=0)
        xmin, xmax = pmin[0], pmax[0]
        ymin, ymax = pmin[1], pmax[1]
        zmin, zmax = pmin[2], pmax[2]

        # create surface grid
        zf = lambda x, y: (n[0]*(x-c[0])+n[1]*(y-c[1])) * -1.0 / n[2] + c[2]
        mgx, mgy = np.meshgrid(
                np.linspace(xmin, xmax),
                np.linspace(ymin, ymax),
            )
        mgz = zf(mgx, mgy)

        # draw
        plt3d = pl.subplot(111, projection='3d')
        plt3d = extend_plt3d(plt3d)

        plt3d.scatter(*np.hsplit(xyz, 3), color='r')
        plt3d.scatter(*np.hsplit(prj, 3), color='g')
        plt3d.plot_surface(mgx, mgy, mgz, color="k", alpha=0.5)

        plt3d.vector(O, c, color='g')
        plt3d.vector(O, C, color='g', alpha=0.5)
        plt3d.vector(c, n+c, color='b')
        plt3d.vector(C, N+C, color='b', alpha=0.5)

        for o, p in zip(xyz, prj):
            plt3d.vector(o, p, color='g', alpha=0.5)

        pl.show()
except ImportError:
    pass

In [ ]:
# n x m dimensional matrix which indicate the orthogonal projection points on the plane

# standard_fit(noAlpha)
print(standard_fit(noAlpha))

plt3d = plt.figure().gca(projection='3d')
plt3d.plot_surface(reshaped, reshaped, reshaped, alpha=0.2)

# Ensure that the next plot doesn't overwrite the first plot
ax = plt.gca()
# ax.hold(True)

ax.scatter(noAlpha[0], noAlpha[1], color='green')

# Height map of pixels on individual channels

## c = centroid
## n = normal vector


$$F(p) = (p-c) \cdot n$$

## p = x, y, r, g, b
## d = direction of channel
[comment]: <> (### pull out coordinates and desired channel)
### p + td = point on the plane
$$F(p+td) = n \cdot (p+td-c) = p \cdot n + t(d \cdot n) - c \cdot n = 0$$
 $$=>  c \cdot n - p\cdot n = t (d \cdot n)$$
 $$=> t = (c \cdot n - p \cdot n)/(d \cdot n)$$

In [ ]:
# # Height map of pixels on idividual channels
# c = 'centroid'
# n = 'normal vector'
# def F(p):
#     return np.dot((p - c), n)
# p = 'point'
# d = 'direction'
# # Pull out coordinates and desired channel
# d = np.zeros((5,), dtype=int)
# d[r] = 1
# # d = [0, 0, 1, 0, 0]
# t = ((np.dot(c,n) - np.dot(p,n))/(np.dot(d,n)))

def getIntPoints(c,n,ch,x,y):
#     d = np.zeros((5,), dtype=int)
#     d = np.zeros((3,))
#     d[ch] = 1
    d = [0.,0.,1.]
#     print(c,n,d)
    pointArray = []
    for i in x:
        for j in y:
            t = 0
            p = [i,j,0]
#             print('c*n=',np.dot(c,n),'p*n=', np.dot(p,n),'d*n=', np.dot(d,n))
            try:
                t = ((np.dot(c,n) - np.dot(p,n))/(np.dot(d,n)))
            except ZeroDivisionError:
                print("Dot product of normal and direction results in zero")
#             print(t,d,t*d[2])
#             tempPoint = p[:,(t)] 
            tempPoint = [p[0],p[1],t] 
            print(tempPoint)
#             print('t=',t,'tempPoint=',tempPoint)
            pointArray.append(tempPoint)
    pA = np.asarray(pointArray)
    xP = pA[:,0]
    yP = pA[:,1]
    dP = pA[:,2]
    display(pA,xP,yP,dP)
    return xP,yP,dP


In [ ]:
print(x,y)

In [ ]:
redLayer = getChannel(reshaped, r)
c,n = (standard_fit(redLayer))
print(c,n)
# display(redLayer)
redCloud = getIntPoints(c,n,r,x,y)

In [ ]:
greenLayer = getChannel(reshaped, g)
c,n = (standard_fit(greenLayer))
print(c,n)
# display(greenLayer)
greenCloud = getIntPoints(c,n,g,x,y)

In [ ]:
blueLayer = getChannel(reshaped, b)
c,n = (standard_fit(blueLayer))
print(c,n)
display(blueLayer)
blueCloud = getIntPoints(c,n,b,x,y)

In [ ]:
display(redCloud,greenCloud,blueCloud)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg

fig = plt.figure()
ax = plt.axes(projection='3d') 

# red plot
rx,ry,rz = redCloud
ax.plot(rx,ry,rz,'.',color='red')

ax.view_init(elev= 10, azim= - 50)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg

fig = plt.figure()
ax = plt.axes(projection='3d') 

# green plot
gx,gy,gz = greenCloud
ax.plot(gx,gy,gz,'^',color='green')

ax.view_init(elev= 10, azim= - 50)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg

fig = plt.figure()
ax = plt.axes(projection='3d') 

# blue plot
bx,by,bz = blueCloud
ax.plot(bx,by,bz,'o',color='blue')

ax.view_init(elev= 10, azim= - 50)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg



fig = plt.figure()
ax = plt.axes(projection='3d') 

# red plot
rx,ry,rz = redCloud
ax.plot(rx,ry,rz,'',color='red')

# green plot
gx,gy,gz = greenCloud
ax.plot(gx,gy,gz,'^',color='green')

# blue plot
bx,by,bz = blueCloud
ax.plot(bx,by,bz,'.',color='blue')

ax.view_init(elev= 10, azim= - 50)
plt.show()

M = greenCloud[:, [0, 1]]
M.show()
# M[:, :][:] = 1

In [ ]:
# Given:
#     data – a set of observed data points
#     model – a model that can be fitted to data points
#     n – minimum number of data points required to fit the model
#     k – maximum number of iterations allowed in the algorithm
#     t – threshold value to determine when a data point fits a model
#     d – number of close data points required to assert that a model fits well to data

# Return:
#     bestfit – model parameters which best fit the data (or nul if no good model is found)
import random

# f(x,y) = Ax + By + C
def MatrixFunction(xyc): # [x, y, c] 
#     f(x,y) = Ax + By + C = [x1, y1, 1] * [A, B, C]T
    p = [xyc[0], xyc[1], 1]

# model = [A, B, C]

def GetError(diff):
    return numpy.dot(diff, diff)

# data is in format [x1, y1, c1]...
def RansacPlaneFit(data, model, n, k, t, d):
    iterations = 0
    bestFit = None
    bestErr = 100000000000000
    for iterations in range(0, k):
        maybeInliers = random.sample(data, n)
        M = maybeInliers[:, [0, 1]]
        M[:, :][:] = 1
#         M = [x1, y1, 1]...
    
#         R = [c1, c2, c3, ...]T
        maybeModel = numpy.linalg.lstsq(M, R) #function that returns [A, B, C]T
        diff = R - M * maybeModel
        alsoInliers = []
        for point in data not in maybeInliers:
            error = GetError(diff)
            if error < t:
                alsoInliers.append(point)
            if len(alsoInliers) > d:
#               this implies that we may have found a good model
#               now test how good it is
                betterModel = maybeModel # model parameters fitted to all points in maybeInliers and alsoInliers
                thisErr = error # a measure of how well model fits these points
                if thisErr < besterr:
                    bestFit = betterModel
                    bestErr = thisErr
    return bestFit




In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import LineModelND, ransac

np.random.seed(seed=1)

# generate coordinates of line
point = np.array([0, 0, 0], dtype='float')
direction = np.array([1, 1, 1], dtype='float') / np.sqrt(3)
xyz = point + 10 * np.arange(-100, 100)[..., np.newaxis] * direction

# add gaussian noise to coordinates
noise = np.random.normal(size=xyz.shape)
xyz += 0.5 * noise
xyz[::2] += 20 * noise[::2]
xyz[::4] += 100 * noise[::4]

# robustly fit line only using inlier data with RANSAC algorithm
model_robust, inliers = ransac(xyz, LineModelND, min_samples=2,
                               residual_threshold=1, max_trials=1000)
outliers = inliers == False

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xyz[inliers][:, 0], xyz[inliers][:, 1], xyz[inliers][:, 2], c='b',
           marker='o', label='Inlier data')
ax.scatter(xyz[outliers][:, 0], xyz[outliers][:, 1], xyz[outliers][:, 2], c='r',
           marker='o', label='Outlier data')
ax.legend(loc='lower left')
plt.show()
%matplotlib inline